# 第三章 字典与集合

dict类型不但在各种程序里广泛使用, 它也是Python语言的基石。 模块的命名空间、实例的属性和函数的关键字参数中都可以看到字典的身影。跟它有关的内置函数都在__builtins__.__dict__ 模块中。

正因为字典至关重要, Python对它的实现做了高度优化, 而散列表则是字典性能出众的根本原因。

同样, 集合(set)的实现其实也依赖于散列表, 因此本章也会讲到它

## 3.1 泛映射类型

In [23]:
from collections import abc
my_dict = {}
isinstance(my_dict, dict),  isinstance(my_dict, abc.Mapping), isinstance(my_dict, abc.MutableMapping)

(True, True, True)

In [24]:
print(abc.Mapping.__doc__)

A Mapping is a generic container for associating key/value
    pairs.

    This class provides concrete generic implementations of all
    methods except for __getitem__, __iter__, and __len__.
    


In [25]:
import collections
import collections.abc
import types
def _isinstance(obj):
    print(f"{type(obj).__name__}", end=" -> ")
    for tp in (collections.abc.Container, 
            collections.abc.Sized, 
            collections.abc.Iterable, 
            collections.abc.Mapping, 
            collections.abc.MutableMapping):
        print(isinstance(obj, tp), end=" ")
    print()

In [26]:
_isinstance({})
_isinstance(collections.ChainMap())
_isinstance(collections.defaultdict())
_isinstance(collections.OrderedDict())
_isinstance(collections.Counter())
_isinstance(types.MappingProxyType({}))     # 不可变映射

dict -> True True True True True 
ChainMap -> True True True True True 
defaultdict -> True True True True True 
OrderedDict -> True True True True True 
Counter -> True True True True True 
mappingproxy -> True True True True False 


### 什么是可散列的数据类型?

In [27]:
tt1 = (1, 2, (30, 40))
tt2 = (1, 2, (30, 40))
tt3 = ((30, 40), 1, 2)
hash(tt1), hash(tt2), hash(tt3)

(-3907003130834322577, -3907003130834322577, -7033943840296421773)

> 注: 两个相同数据内容的哈希映射唯一

In [28]:
t1 = (1, 2, [30, 40])
hash(t1)

TypeError: unhashable type: 'list'

> 注: 对于元组, 只有其包含的全部元素都是可散列类型时, 它才是可散列的

In [29]:
class Preson:
    pass
preson = Preson()
hash(preson),  hash(id(preson))

(135740654777, 2171850476432)

> 一般来说, 用户自定义的类型都是可以散列的

## 3.2 字典推导

In [30]:
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
]

In [31]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [32]:
country_code = {code:country.upper() for code, country in DIAL_CODES if code <= 66}
country_code

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

## 3.3 常见的映射方法

#### collections.defaultdict

In [33]:
from collections import defaultdict

# 统计字符出现的次数
text = "hello world"
# char_count = defaultdict(int)
char_count = defaultdict(lambda: -1)    # 设置默认值
for char in text:
    char_count[char] += 1
print(char_count)

defaultdict(<function <lambda> at 0x000001F9AC959A80>, {'h': 0, 'e': 0, 'l': 2, 'o': 1, ' ': 0, 'w': 0, 'r': 0, 'd': 0})


In [34]:
char_count['s']

-1

* defaultdict 的优点
    * 避免 KeyError 异常： 在访问不存在的键时，会自动提供默认值。
    * 简化代码： 可以避免大量的 if-else 判断。
    * 灵活的默认值： 可以自定义默认值工厂，满足各种需求。
* defaultdict 的常见用法
    * 计数： 统计元素出现的次数。
    * 分组： 将数据按照某个属性分组。
    * 构建多层字典： 创建嵌套的字典结构。

### collections.OrderedDict

In [35]:
from collections import OrderedDict

# 创建一个 OrderedDict
ordered_dict = OrderedDict()

# 添加键值对
ordered_dict['a'] = 1
ordered_dict['b'] = 2
ordered_dict['c'] = 3

# 访问值
print(ordered_dict['a'])  # 输出：1

# 迭代
for key, value in ordered_dict.items():
    print(key, value)

1
a 1
b 2
c 3


* OrderedDict 是一个非常有用的数据结构，它在需要保持键序的场景下非常有用。
* 通过合理地使用 OrderedDict，可以实现很多有趣的功能，比如 LRU 缓存、自定义数据结构等。

### 用setdefault处理找不到的键

In [36]:
country_code.setdefault(22, "E")
country_code

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA', 22: 'E'}

In [37]:
country_code.setdefault(23, {})['H'] = 3
country_code

{1: 'UNITED STATES',
 62: 'INDONESIA',
 55: 'BRAZIL',
 7: 'RUSSIA',
 22: 'E',
 23: {'H': 3}}

* 使用该方法进行查找更新, 通常比通过if判断查询少很多次查询次数, 通常只需要查询一次即可

In [38]:
# 使用 if 方式进行键查询并更新
if 23 not in country_code:
    country_code[23] = {}
country_code[23]['e']  = 3
country_code

{1: 'UNITED STATES',
 62: 'INDONESIA',
 55: 'BRAZIL',
 7: 'RUSSIA',
 22: 'E',
 23: {'H': 3, 'e': 3}}

## 3.4 映射的弹性键查询

### defaultdict: 处理找不到的键的一个选择

### 特殊方法 __missing__

所有的映射类型在处理找不到的键的时候, 都会牵扯到 __missing__ 方法

In [39]:
class StrKeyDict0(dict): 
    def __missing__(self, key):
        if isinstance(key, str):  
            raise KeyError(key)
        return self[str(key)]  
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        # return key in self or str(key) in self  # 使用更Python的写法, 会导致递归调用 __contains__ 
        return key in self.keys() or str(key) in self.keys()  

In [40]:
d = StrKeyDict0([('2', 'two' ), ('4', 'four')])
d['2'], d['4']

('two', 'four')

In [41]:
# missing 方法将数字2 转为了字符串
d[2]

'two'

In [42]:
d[1]    # 对于不存在情况抛出异常

KeyError: '1'

In [43]:
2 in d

True

## 3.5 字典的变种

* collections.OrderedDict
* collections.ChainMap
* collections.Counter
* collections.UserDict

### collections.ChainMap

In [44]:
import builtins
from collections import ChainMap

pylookup = ChainMap(locals(), globals(), vars(builtins))
pylookup

ChainMap({'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'import collections\nimport collections.abc\ndef _isinstance(obj):\n    for tp in (collections.abc.Container, \n            collections.abc.Sized, \n            collections.abc.Iterable, \n            collections.abc.Mapping, \n            collections.abc.MutableMapping):\n        print(isinstance(obj, tp), end=" ")\n    print()', 'import collections\nimport collections.abc\ndef _isinstance(obj):\n    print(f"{obj!r}: {type(obj)}", end=" ")\n    for tp in (collections.abc.Container, \n            collections.abc.Sized, \n            collections.abc.Iterable, \n            collections.abc.Mapping, \n            collections.abc.MutableMapping):\n        print(isinstance(obj, tp), end=" ")\n    print()', '_isinstan

### collections.Counter

&emsp; 利用Counter 来计算单词中各个字母出现的次数

In [45]:
from collections import Counter
ct = Counter('abababarrandab')
ct

Counter({'a': 6, 'b': 4, 'r': 2, 'n': 1, 'd': 1})

### collections.UserDict

这个类实际是把标准的dict用Python重新实现了一遍。UserDict 是让用户继承写子类的

## 3.6 子类化UserDict

&emsp; 就创造自定义映射类型而言, 以UserDict为基类, 总比以普通的dict为基类要来的方便

In [46]:
from collections import UserDict

class StrKeyDict(UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [47]:
sk = StrKeyDict(name="Rookie", age=25)
sk[1] = [0, 1, 2]   # __setitem__方法将数字键转为字符键
sk, sk[1]

({'name': 'Rookie', 'age': 25, '1': [0, 1, 2]}, [0, 1, 2])

In [48]:
sk.get('name', "lambda")    # Mapping 类中实现的get方法和StrKeyDict0 中一致, 而UserDict的get方法继承自Mapping

'Rookie'

## 3.7 不可变映射类型 

In [49]:
from types import MappingProxyType
d = {1:['A', 'N']}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: ['A', 'N']})

In [50]:
d_proxy[1]

['A', 'N']

In [51]:
d_proxy[2] = 12

TypeError: 'mappingproxy' object does not support item assignment

In [52]:
d_proxy[1].append('D')
d_proxy

mappingproxy({1: ['A', 'N', 'D']})

## 3.8 集合论

In [53]:
l = [2, 4, 4, 2, 0, 1, 0]
set1 = {2, 0, -1}
set2 = set(l)
set1, set2

({-1, 0, 2}, {0, 1, 2, 4})

In [54]:
set1 & set2, set1 | set2

({0, 2}, {-1, 0, 1, 2, 4})

### 3.8.1 集合字面量 

* 句法的陷阱:
    * 如果要创建一个空集, 你必须用不带任何参数的构造方法set(). 如果只是写成{}的形式, 跟以前一样, 你实际创建的是一个空字典

In [55]:
s1 = set()
s2 = {}
type(s1), type(s2)

(set, dict)

In [56]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

### 3.8.2 集合推导

In [57]:
from unicodedata import name
{chr(i) for i in range(32, 256) if "SIGN" in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### 集合的操作 

In [58]:
s1 = {12, 3, 4,  2}
s2 = {0, 12, 2, 4, 10}
s2 - s1, s1 - s2, s1^s2

({0, 10}, {3}, {0, 3, 10})

In [59]:
s1 = {1, 2}
s2 = {x for x in range(20)}
s2 <= s2, s1 < s2, s1 > s2 

(True, True, False)

## 3.9 dict 和 set 的背后

### dict的实现及其导致的结果

* 01. 键必须是可散列的
* 02. 字典的内存开销巨大
* 03. 键的查询很快
* 04. 键的次序取决于添加的顺序
* 05. 往字典里添加新键可能会改变已有键的顺序


### set的实现及其导致的结果

* 和dict类似

## 3.10 本章小结

<img src="./images/第三章总结.jpg" width="70%">